In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [5]:
design.delete_all_components()

In [25]:
design.get_chip_z('Q_chip')

'7um'

In [3]:
design.chips['C_chip'].size['size_x']='6mm'
design.chips['C_chip'].size['size_y']='6mm'
design.chips['Q_chip'].size['size_x']='4mm'
design.chips['Q_chip'].size['size_y']='4mm'
design.chips['Q_chip'].size['center_z']='7um'

# Resonator length

In [4]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [5]:
resonator_length_4=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,4)
    a=a.strip('mm')
    a=float(a)-0.35+0.51-0.1+0.4-0.2-0.1
    
    resonator_length_4.append(a)
print(resonator_length_4)


[4.386627341935143, 4.2723941699207115, 4.164173270104566]


# Flipchip

In [6]:
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))

# q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
#                                             pos_y='-2000um',
#                                             chip='Q_chip',
#                                             connection_pads=cp0))]
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-1300um',
                                            pos_y='-470um',
                                            chip='Q_chip',
                                            pad_width='355um',
                                            pad_height='65um',
                                            pad_gap='50um',
                                            pocket_width='650um',
                                            pocket_height='650um'))]

qubits=q0

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = StraightUpCoupler(design, "c0", options=dict(pos_x=f"{q0x-100}um",
                                              pos_y=f"{q0y+60}um",
                                              pad_width="150um",
                                              pad_height="65um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))

design.rebuild()
gui.rebuild()
gui.autoscale()

In [7]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-1700um',
                                         pos_y = '1300um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '1700um',
                                         pos_y = '1300um',
                                         orientation='180',
                                         lead_length = '30um'))

In [8]:


stg=ShortToGround(design, 'stg', options=Dict(chip="C_chip",
                                                pos_x='-0.585mm',
                                                pos_y='0mm',
                                                orientation='270'))

In [9]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [10]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [11]:
jogs1=OrderedDict()
jogs1[0]=['R','350um']
jogs1[1]=['R', '400um']
jogs1[2]=['L','465um']


In [12]:
#3.91mm
rr0=connect('rr0', 'c0', 'in','stg', 'short', 
            resonator_length_4[1],jogs1,jogs, 
            '400um','0um', '1460um', '100um')

# EPR Analysis

In [13]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [14]:
hfss.start()

INFO 06:31PM [connect_project]: Connecting to Ansys Desktop API...
INFO 06:31PM [load_ansys_project]: 	Opened Ansys App
INFO 06:31PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 06:31PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   flipchip_distance
INFO 06:31PM [connect_design]: 	Opened active design
	Design:    flip8um_V5 [Solution type: Eigenmode]
INFO 06:31PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 06:31PM [connect]: 	Connected to project "flipchip_distance" and design "flip8um_V5" 😀 



True

In [15]:
hfss.activate_ansys_design("flip7um_V5--", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','stg','rr0','fl0'],[],
                    box_plus_buffer=False)

06:31PM 39s WARNING [activate_ansys_design]: The design_name=flip7um_V5-- was not in active project.  Designs in active project are: 
['flip8um_V5', 'flip6um_V5', 'Design_q3d'].  A new design will be added to the project.  
INFO 06:31PM [connect_design]: 	Opened active design
	Design:    flip7um_V5-- [Solution type: Eigenmode]
WARNING 06:31PM [connect_setup]: 	No design setup detected.
WARNING 06:31PM [connect_setup]: 	Creating eigenmode default setup.
INFO 06:31PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [16]:
hfss.close()

True

# LOM Analysis

In [26]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")


In [27]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.88',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.88',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [29]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 05:13PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp34gajm9x.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5880000000, Maxwell, 1, False
INFO 05:13PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpbwu00vkm.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5880000000, Maxwell, 1, False
INFO 05:13PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpd0pgen8t.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5880000000, Maxwell, 2, False
INFO 05:13PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp3d4dhihy.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5880000000, Maxwell, 3, False
INFO 05:13PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpplqar11w.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5880000000, Maxwell, 4, False
INFO 05:13

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,293.510,-84.7530,-69.981,-78.3030
pad_bot_Q0,-84.753,99.8070,-13.338,-0.1456
pad_top_Q0,-69.981,-13.3380,99.726,-14.7060
quad_coupler_c0,-78.303,-0.1456,-14.706,93.1600


In [30]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 6.92
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.959039 [GHz]
EC 335.506219 [MHz]
EJ 14.854145 [GHz]
alpha -389.693848 [MHz]
dispersion 36.460045 [KHz]
Lq 10.995564 [nH]
Cq 57.734334 [fF]
T1 16.759400 [us]

**Coupling Properties**

tCqbus1 -7.289466 [fF]
gbus1_in_MHz -121.649633 [MHz]
χ_bus1 -8.958087 [MHz]
1/T1bus1 9496.458177 [Hz]
T1bus1 16.759400 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,6.348955,384.317035,14.854145,-452.687649,119.531828,[-131.09311825271442],[-26.70684035109583],26.706840,131.093118
2,5.973578,337.257938,14.854145,-391.927091,38.218758,[-136.70123013831287],[-11.6553502461473],11.655350,136.701230
3,5.989301,339.158157,14.854145,-394.351942,40.205707,[-131.2824645466438],[-11.106775487138732],11.106775,131.282465
4,5.978968,337.90869,14.854145,-392.757241,38.88981,[-126.45758890387347],[-10.085990860223722],10.085991,126.457589
5,5.971526,337.010414,14.854145,-391.611404,37.966053,[-123.70951906996433],[-9.504866256018877],9.504866,123.709519
6,5.971303,336.983539,14.854145,-391.57713,37.9387,[-122.94408108591979],[-9.383293042073223],9.383293,122.944081
7,5.966259,336.375447,14.854145,-390.801763,37.324137,[-121.36778334112579],[-9.049679454427507],9.049679,121.367783
8,5.960785,335.716337,14.854145,-389.961619,36.667382,[-120.54563069426989],[-8.827739897508248],8.827740,120.545631
9,5.959039,335.506219,14.854145,-389.693848,36.460045,[-121.6496334229745],[-8.958087295111714],8.958087,121.649633


In [17]:
c1.sim.close()

NameError: name 'c1' is not defined